## Without using Pipeline - 

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [29]:
df = pd.read_csv("train.csv")
df.head(2)

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   

   Parch     Ticket     Fare Cabin Embarked  
0      0  A/5 21171   7.2500   NaN        S  
1      0   PC 17599  71.2833   C85        C

In [30]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace=True)

In [32]:
df.head(2)

Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C

In [33]:
X = df.drop(['Survived'], axis = 1)
X.head(1)

Pclass   Sex   Age  SibSp  Parch  Fare Embarked
0       3  male  22.0      1      0  7.25        S

In [37]:
y = df['Survived']
y.head(1)

0    0
Name: Survived, dtype: int64

In [38]:
df.shape

(891, 8)

In [123]:
X_train,X_test,y_test,y_train = train_test_split(X,y, train_size=0.8, random_state=10, stratify=y)

In [124]:
X_train.shape

(712, 7)

In [125]:
X_test.shape

(179, 7)

In [126]:
y_train.shape

(179,)

In [127]:
y_test.shape

(712,)

In [128]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

* here we can see that out of all the columns, 'Age' column has 177 missing values and 'Embarked' column has 2 missing values
* since age is a numerical column and embarked is categorical in nature so we will replace age by its mean (mean of the entire age column) and embarked with the 'most frequent' value
* as for the age column we are applying the 'mean' method for the missing values and 'mode(most frquent)' for the embarked column so we will create two separate objects of the 'Imputer' class as shown below-

#### replacing the missing values of the 'Age' and 'Embarked' column - 

In [129]:
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

In [130]:
X_train_age = si_age.fit_transform(X_train[['Age']])

In [131]:
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

In [132]:
X_test_age = si_age.fit_transform(X_test[['Age']])
X_test_embarked = si_embarked.fit_transform(X_test[['Embarked']])

In [133]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

#### Applying One-Hot encoding - 

In [134]:
df.head(2)

Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C

In [135]:
## Applying one hot encoding on 'Sex' and 'Embarked' column - 

In [136]:
ohe_gender = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
ohe_embarked = OneHotEncoder(sparse=False, handle_unknown='ignore')

* here, 'handle_unknown' means that if in future we get the new category of variables then those values will be ignored with zero.

In [137]:
X_train_sex = ohe_gender.fit_transform(X_train[['Sex']])

In [138]:
X_train_embarked = ohe_embarked.fit_transform(X_train[['Embarked']])

In [139]:
X_test_sex = ohe_gender.fit_transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.fit_transform(X_test[['Embarked']])

In [140]:
X_train.head(1)

Pclass     Sex  Age  SibSp  Parch  Fare Embarked
415       3  female  NaN      0      0  8.05        S

In [141]:
X_train_embarked

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

* here, multi-collinearity is not eliminated (through - 'drop = 'first'')

* so now we can use that we have dofferent arrays for - 
    * sex
    * embarked
    * age
* we will now concatenate all of arrays

In [142]:
X_train_remaining = X_train.drop(['Age', 'Sex', 'Embarked'], axis=1)

In [143]:
X_test_remaining = X_test.drop(['Age', 'Sex', 'Embarked'], axis = 1)

In [144]:
X_test_remaining.head(2)

Pclass  SibSp  Parch     Fare
24        3      3      1  21.0750
251       3      1      1  10.4625

In [145]:
X_train_remaining.head(2)

Pclass  SibSp  Parch  Fare
415       3      0      0  8.05
131       3      0      0  7.05

In [146]:
X_train.head(1)

Pclass     Sex  Age  SibSp  Parch  Fare Embarked
415       3  female  NaN      0      0  8.05        S

In [147]:
df.head(1)

Survived  Pclass   Sex   Age  SibSp  Parch  Fare Embarked
0         0       3  male  22.0      1      0  7.25        S

In [148]:
## now applying concatenation - 

In [149]:
X_train_transformed = np.concatenate((X_train_remaining,X_train_age,X_train_embarked,X_train_sex), axis = 1)
X_test_transformed = np.concatenate((X_test_remaining,X_test_age,X_test_embarked,X_test_sex), axis = 1)

In [150]:
X_train_embarked.shape

(712, 4)

In [151]:
X_test_embarked.shape

(179, 3)

In [152]:
X_test_age.shape

(179, 1)

In [153]:
X_train_transformed

array([[3., 0., 0., ..., 0., 1., 0.],
       [3., 0., 0., ..., 0., 0., 1.],
       [1., 1., 2., ..., 0., 1., 0.],
       ...,
       [3., 3., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [3., 1., 0., ..., 0., 0., 1.]])

In [154]:
X_train_transformed.shape

(712, 11)

In [155]:
X_train.shape

(712, 7)

In [156]:
X_test_transformed.shape

(179, 10)

In [157]:
import pickle

*** end of this notebook***